In [1]:
!ls data/amazon-reviews/

baby-products.csv        grocery-gourmet-food.csv toys-games.csv
beauty.csv               health-personal-care.csv train_40k.csv
cat1.csv                 pet-supplies.csv         val_10k.csv


# Top level classifier

In [11]:
from sklearn import *
import csv
import pandas as pd

In [9]:
from sklearn.pipeline import Pipeline


In [28]:
clf = Pipeline(
    steps=[
        ('tok', sklearn.feature_extraction.text.TfidfVectorizer(min_df = 3, max_df=0.8)),
        ('clf', sklearn.svm.LinearSVC(penalty='l2', multi_class='ovr', class_weight='balanced'))
    ]
)

In [17]:
df1 = pd.read_csv('data/amazon-reviews/cat1.csv', index_col=False); df1.head()

,Unnamed: 0,Cat1,text_feature
0,0,grocery gourmet food,Golden Valley Natural Buffalo Jerky The descri...
1,1,toys games,Westing Game This was a great book!!!! It is w...
2,2,toys games,"Westing Game I am a first year teacher, teachi..."
3,3,toys games,Westing Game I got the book at my bookfair at ...
4,4,toys games,I SPY A is For Jigsaw Puzzle 63pc Hi! I'm Mart...


In [21]:
X = list(df1['text_feature'])
Y = list(df1['Cat1'])
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X,Y, train_size=0.8)

In [22]:
assert len(X_train) == len(Y_train)

In [29]:
clf.fit(X_train, Y_train)

Pipeline(steps=[('tok', TfidfVectorizer(max_df=0.8, min_df=3)),
                ('clf', LinearSVC(class_weight='balanced'))])

In [30]:
Y_pred = clf.predict(X_test)
print(sklearn.metrics.classification_report(Y_pred, Y_test))

                      precision    recall  f1-score   support

       baby products       0.95      0.94      0.94      1143
              beauty       0.94      0.92      0.93      1183
grocery gourmet food       0.93      0.92      0.93       707
health personal care       0.91      0.92      0.92      1935
        pet supplies       0.96      0.96      0.96       925
          toys games       0.96      0.97      0.97      2107

            accuracy                           0.94      8000
           macro avg       0.94      0.94      0.94      8000
        weighted avg       0.94      0.94      0.94      8000



- [ ] Look at examples where model is incorrect - might tell us about the data
- [ ] Create loop that train models for each sub category